In [1]:
deployment = "gpt4"
model = "gpt-4"

# 回顾

In [9]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMRequestsChain
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

# https://www.cnblogs.com/sunsky303/p/11045113.html

def query_web(question):
      template = """Between >>> and <<< are the raw search result text from google.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:"""

      PROMPT = PromptTemplate(
          input_variables=["query", "requests_result"],
          template=template,
      )

      inputs = {
          "query": question,
          #"url": "https://cn.bing.com/search?q=" + question.replace(" ", "+").replace("?","")
          "url": "https://www.baidu.com/s?wd=" + question.replace(" ", "+").replace("?","")
      }
      requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info")
      res = requests_chain.run(inputs)
      print(f"Result:{res}")
      return res

In [10]:
import os

# os.environ["BING_SUBSCRIPTION_KEY"] = "<Your Key>"
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

from langchain.utilities import BingSearchAPIWrapper

def query_web(question):
    search = BingSearchAPIWrapper()
    return search.run(question)


In [11]:
def get_current_cluster_state(cluster_name):
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

In [12]:
funcs = {
    "get_current_cluster_state": get_current_cluster_state,
    "query_web":query_web,
}

In [13]:
import openai
import json
def run(input):
    msg=[{"role":"user","content":input}]
    ret = run_conversation(msg)
    return ret["content"] 

def run_conversation(msg):
    response = openai.ChatCompletion.create(
        engine=deployment,
        model=model,
        messages=msg,
        functions=[
     
            {
                "name": "get_current_cluster_state",
                "description": "Get the current state in a given cluster",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cluster_name": {
                            "type": "string",
                            "description": "the name of the cluster",
                        },
                        
                    },
                    "required": ["cluster_name"],
                },
            },
     
            {
                "name": "query_web",
                "description": "Get a realtime info/unknown info from web",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "the query/question",
                        },
                        
                    },
                    "required": ["query"],
                },
            }
        ],
        function_call="auto",
    )

    message =  response["choices"][0]["message"]
    
    # 如果不需要调用function，则直接返回结果
    if not message.get("function_call"):
        return message
    
    # 从
    function_name = message["function_call"]["name"]
    function_args = json.loads(message["function_call"]["arguments"])
    print(function_args)
    res = funcs[function_name](function_args["question"])
    message["content"]=None
    msg.append(message)
    msg.append({
                "role": "function",
                "name": function_name,
                "content": res,
            })
    return run_conversation(msg)

In [14]:
run("北京今天适合穿什么")

{'question': '北京今天的天气如何'}


'北京今天的天气是晴天，温度在19度到29度之间。建议您着装轻薄，可以穿短袖、短裤或是裙子，并配备一件薄外套以防晚上温度下降。外出时要注意防晒保护工作。'

# LangChain Agent

In [15]:
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMRequestsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import time


In [16]:
tools = [
    Tool(
        name = "Get current info",func=query_web,
        description="""invoke it for when you need to answer questions about current info.
             And the input should be a search query."""),
    Tool(
        name="Get current cluster state", func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given cluster, 
        And the input should be service name.""")
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", max_iterations=3, verbose=True)

In [17]:
agent.run("北京今天穿什么合适？")



> Entering new AgentExecutor chain...
我需要获取当前的天气信息来回答这个问题。
Action: Get current info
Action Input: 北京今天的天气
Observation: <b>今天</b> 7天 8-15天 40天 雷达图 31日（<b>今天</b>） 晴 29 / 19℃ &lt;3级 1日（明天） 多云 30 / 21℃ &lt;3级 2日（后天） 小雨 28 / 20℃ &lt;3级 3日（周日） 小雨 28 / 19℃ &lt;3级 4日（周一） 多云 29 / 20℃ &lt;3级 5日（周二） 晴 30 / 21℃ &lt;3级 6日（周三） 晴 31 / 21℃ &lt;3级 分时段预报 生活指数 蓝天预报 <b>天气</b>资讯 全国开学<b>天气</b>地图出炉 北方大部秋高气爽广东福建开学“撞”台风 中国<b>天气</b>网 2023-08-31 11:20 湖南大部<b>今天</b>阳光在线 明晚起南部将有较强降雨过程局地暴雨 中国<b>天气</b>网湖南站 2023-08-31 11:10 【<b>北京天气</b>】<b>北京天气</b>预报,<b>天气</b>预报一周,<b>天气</b>预报15天查询 <b>北京</b> &gt; 城区 <b>今天</b> 7天 8-15天 更长时间 雷达图 29日（<b>今天</b>） 霾转晴 4 / -4℃ 微风 30日（明天） 晴 5 / -6℃ 3-4级转微风 31日（后天） 晴转多云 4 / -6℃ 微风 1日（周五） 多云转晴 4 / -4℃ 微风 2日（周六） 晴转多云 5 / -4℃ 3-4级转微风 3日（周日） 多云转晴 4 / -4℃ 微风 4日（周一） 晴转多云 4 / -2℃ 微风 分时段预报 生活指数 蓝天预报 周边地区 | 周边景点 2023-08-31 18:00更新 香河 / 7天天气预报（2023/08/30 08:00发布）. 星期三. 08/30. 多云. 无持续风向. 微风. 29℃. 19℃. 多云. 日出05:39. 北京. 日落18:51. 降水量. 相对湿度. 体感温度. 雷达图. 发布时间：08-30 12:00. 08/30. 大部晴朗. Feels-Like Temperature.

'今天北京的天气晴朗，温度在29℃到19℃之间，风力小于3级。建议穿着短袖或者薄款长袖，搭配一条舒适的裤子，早晚温度较低时可以加一件薄外套。'

In [11]:
agent.run("如何修复的ADX集群？")



> Entering new AgentExecutor chain...
首先，我需要了解ADX集群的当前状态。
Action: Get current cluster state
Action Input: ADX集群
Observation: ERROR: Failed to pull image "chaocai/docker/dsp:latest
Thought:看来ADX集群的问题在于无法拉取最新的docker镜像。我需要查找如何解决这个问题。
Action: Get current info
Action Input: 如何修复无法拉取docker镜像的问题Question:如何修复无法拉取docker镜像的问题
Result:not found

Observation: not found
Thought:我没有找到具体的解决方案，但通常，无法拉取docker镜像的问题可能是由于网络问题或者docker仓库的问题。我可以建议一些可能的解决步骤。
Final Answer: 无法拉取docker镜像的问题可能是由于网络问题或者docker仓库的问题。你可以尝试以下步骤来解决这个问题：
1. 检查你的网络连接。
2. 确保你有权限访问docker仓库。
3. 检查docker仓库是否存在问题或者是否有最新的镜像。
4. 如果以上步骤都无法解决问题，你可能需要联系docker仓库的管理员或者寻求专业的技术支持。

> Finished chain.


'无法拉取docker镜像的问题可能是由于网络问题或者docker仓库的问题。你可以尝试以下步骤来解决这个问题：\n1. 检查你的网络连接。\n2. 确保你有权限访问docker仓库。\n3. 检查docker仓库是否存在问题或者是否有最新的镜像。\n4. 如果以上步骤都无法解决问题，你可能需要联系docker仓库的管理员或者寻求专业的技术支持。'

# 利用LangChain中的工具

In [20]:
pip install wolframalpha

  Using cached wolframalpha-5.0.0-py3-none-any.whl (7.5 kB)
  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
  Using cached jaraco.context-4.3.0-py3-none-any.whl (5.3 kB)
Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
def wolfram_api(question):
    wolfram = WolframAlphaAPIWrapper()
    return wolfram.run(question)

In [22]:
wolfram_api("What is 2x+5 = -3x + 7?")

'Assumption: 2 x + 5 = -3 x + 7 \nAnswer: x = 2/5'

In [23]:
tools = [
    Tool(
        name = "Get current info",func=query_web,
        description="""invoke it for when you need to answer questions about current info.
             And the input should be a search query."""),
    Tool(
        name="Get current cluster state", func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given cluster, 
        And the input should be service name."""),
    Tool(
        name="Invoke WolframAlpha API", func=wolfram_api,
        description="""invoke it for when you need to anwser or solve a mathematical question, 
        And the input is the question""")
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", max_iterations=3, verbose=True)

In [24]:
agent.run("在一个外星球上，这里的鸡都长着3只脚1个头，免子有4只脚1个头，一个装着鸡和免子的笼子里一共有20个头，71只脚，请问笼子里各有多少只鸡和免子？")



> Entering new AgentExecutor chain...
这是一个数学问题，可以通过设置方程来解决。设鸡的数量为x，免子的数量为y。那么我们可以得到两个方程：x + y = 20 (头的数量) 和 3x + 4y = 71 (脚的数量)。我可以使用WolframAlpha API来解决这个问题。
Action: Invoke WolframAlpha API
Action Input: Solve the equations x + y = 20 and 3x + 4y = 71
Observation: Assumption: solve x + y = 20
3 x + 4 y = 71 
Answer: x = 9 and y = 11
Thought:我现在知道了最后的答案
Final Answer: 笼子里有9只鸡和11只免子。

> Finished chain.


'笼子里有9只鸡和11只免子。'